# STEP
1. Look for `SMS in progress to true` in **verbose.txt** click send button in desktop [Timestamp] 
2. Look for `Sending wakeup PNS request` in **verbose.txt**
3. Grab the timestamp from the request and look for it in **sync.txt** 
    - Timestamp can be matched by deleting last value from milis part
    - Grab the request id and look for the request response code [CorrelationId]
4. Request code success: 200, failed: 400... [Timestamp]
5. If request is success look for correlationId in **mav_sdk.txt**
    - CorrectionId can be found in pns for `DeviceDiscoveryNotification`
6. Now OTT will start bluetooth advertising and send packet to MDNS 
    - Bluetooth advertising status: `Advertising onStartSuccess`, `Advertising onStartFailure`  in **MavenirUCC.txt** [Timestamp]
    - Packet sent to MDNS: `MavNsdHelper onServiceRegistered` in **mav_sdk.txt** [Timestamp]
        + Need to include correlationId/hash to compare [Improvement]
7. Package received by Desktop: [Status] 
    - Bluetooth: look for CorrelationId in **discovery.txt** [Timestamp]
    - MDNS: `mdnsTraces:: wifiMatch string` with hash key in **discovery.txt** [Timestamp]
8. Total round-trip time: from **step 1** to **step 7**
9. Overall SMS [Status]


In [32]:
# Import
import pandas as pd
import re
from pandas_datareader import data
import matplotlib.pyplot as plt
import os
import itertools
from datetime import datetime

# Config
configDate = "2023-11-14"
configDateFormatter = "%Y-%m-%d"

dateTotalTimeFormatter = '%H:%M:%S.%f'

logDesktopDir = "../log-analysis/logs/desktop/"
logOTTDir = "../log-analysis/logs/mobile_android/"

logsDateFormatter = {
    "verbose": ("%d/%m/%Y", [7, 17]),  # 14/11/2023
    "sync": ("%d/%m/%Y", [8, 18]), # 14/11/2023
    "discovery": ("%d/%m/%Y", [0, 10]), # 14/11/2023
    "mav_sdk":("%Y-%m-%d", [1, 11]), # 2023-11-14 
    "MavenirUCC": ("%Y-%m-%d", [0, 5]) # 11-14 :fixing by adding year manually
}

In [33]:
# Output
cDate = 'Date'
cCoId = 'Co-Relation ID'
cSMSSent = 'SMS Sent from UI (Desktop)'
cWakeupPNSSent = 'Wake up PNS Sent to WRG (Desktop)'
cWakeupPNSReceived = 'Wake up PNS Received (OTT)'
cPacketBroadcast = 'Starting broadcast MDNS/BLE Packet (OTT)'
cPacketReceivedStatus = 'Packet Received (Desktop) \u274c or \u2705'
cPacketReceivedTime = 'Packet Received Time (Desktop)'
cTotalTime = 'Total Round-Trip Time'
cSMSStatus = 'Overall SMS Status \u274c or \u2705'

cols = [
    cDate,
    cCoId,
    cSMSSent,
    cWakeupPNSSent,
    cWakeupPNSReceived,
    cPacketBroadcast,
    cPacketReceivedStatus,
    cPacketReceivedTime,
    cTotalTime,
    cSMSStatus
]
df = pd.DataFrame(None, columns=cols) 
df.fillna(-1)

# Patterns
smsStart = 'SMS in progress to true'
wakeUpPNSRequest = 'Sending wakeup PNS request'

# Color
failed = u'\u274c'
success = u'\u2705'



In [34]:
# Utilities
def _datetime(date_str, formatter):
    return datetime.strptime(date_str, formatter)

def _dirList(fileName, dirPath):
    res = []
    files = os.listdir(dirPath)
    for f in files:
        if fileName in f:
            # Verify the date
            filePath = dirPath + f
            with open(filePath) as file:
                data = [line.rstrip() for line in file]
                start = 0
                end = len(data) - 1
                isAfter = 0
                isBefore = 0
                while True:
                    try:
                        startLine = data[start]
                        # print(startLine)
                        if startLine.strip() != "":
                            if _isDateAfter(startLine, f) == True:
                                isAfter += 1
                            else: 
                                isAfter -= 1
                            break
                        else:
                            start+=1
                    except:
                        start+=1
                while True:
                    try :    
                        endLine = data[end]
                        # print(endLine)
                        if endLine.strip() != "":
                            if _isDateBefore(endLine, f) == True:
                                isBefore += 1
                            else: 
                                isBefore -= 1
                            break
                        else:
                            end-=1
                    except:
                        end-=1

                if isAfter == 1 and isBefore == 1:
                    # print("Appending this file: " + f)
                    res.append(filePath)
    return res

def _isDateAfter(line, fileName):
    tmp = dict(filter(lambda x: x[0] in fileName, logsDateFormatter.items()))
    logsType = list(tmp.values())[0]
    formatter = logsType[0]
    indexes = logsType[1]
    if str(line).startswith("debug") and "verbose" in fileName:
        indexes = [8, 18]
    if "MavenirUCC" in str(fileName):
        date = "2023-" + line[indexes[0]: indexes[1]]
    else:
        date = line[indexes[0]: indexes[1]] 
    # print("Start date extracted: " + date)
    logStateDate = _datetime(date, formatter)
    # print(logStateDate)
    inputDate = _datetime(configDate, configDateFormatter)
    # print(inputDate)
    return inputDate >= logStateDate

def _isDateBefore(line, fileName):
    tmp = dict(filter(lambda x: x[0] in fileName, logsDateFormatter.items()))
    logsType = list(tmp.values())[0]
    formatter = logsType[0]
    indexes = logsType[1]
    if str(line).startswith("debug") and "verbose" in fileName:
        indexes = [8, 18]
    if "MavenirUCC" in str(fileName):
        date = "2023-" + line[indexes[0]: indexes[1]]
    else:
        date = line[indexes[0]: indexes[1]] 
    # print("End date extracted: " + date)
    logStateDate = _datetime(date, formatter)
    inputDate = _datetime(configDate, configDateFormatter)
    return inputDate <= logStateDate

files_verbose = _dirList("verbose", logDesktopDir)
files_discovery = _dirList("discovery", logDesktopDir)
files_sync = _dirList("sync", logDesktopDir)
files_mav_sdk = _dirList("mav_sdk", logOTTDir)
files_mavnir_ucc = _dirList("MavenirUCC", logOTTDir)



In [35]:
# Step 1 & 2
print("\n========== Step 1 & 2 ==========")
dWakeupPNSRequestTime = []
dfIndex = 0
for f in files_verbose:
    print("--- Reading file:", f)
    with open(f) as file:
        verbose = [line.rstrip() for line in file]
        for index, line in enumerate(verbose):
            if smsStart in line: 
                df.at[dfIndex, cDate] = configDate
                df.at[dfIndex, cSMSSent] = line[18:30]
                dfIndex+=1
            if wakeUpPNSRequest in line:
                print(line[18:30])
                dWakeupPNSRequestTime.append(line[18:30])



========== Step 1 & 2 ==========
--- Reading file: ../feedback-analysis/logs/desktop/verbose.txt
11:11:40.694
11:13:24.844
11:13:55.449
11:15:05.788
11:18:19.952
11:19:44.671
11:22:04.121
11:24:12.012
11:25:07.924


In [36]:
# Step 3 & 4 
print("\n========== Step 3 & 4 ==========")
dCoId = {}
dfIndex = 0
for f in files_sync:
    print("--- Reading file:", f)
    with open(f) as file:
        sync = [line.rstrip() for line in file]
        for index, timestamp in enumerate(dWakeupPNSRequestTime):
            status = False
            coId = ""
            log = []
            count = 0
            # print(timestamp)
            while (1):
                count+=1
                l = len(timestamp)
                log = list(filter(lambda x: x.find(timestamp) != -1, sync))
                if len(log) > 0:
                    break
                timestamp = timestamp[0:l-1]
                if count == 3:
                    break
                
            # print("Looking for time: ", l)
            for l in log: 
                # print("processing log: " + l)
                u = "[uuid]=>"
                if u in l:
                    id = l[l.index(u) + len(u):l.index(" [readyState]")]
                    print(id)
                    for r in list(filter(lambda x: x.find(id) != -1, sync)):
                        start = sync.index(r)
                        end = start + 65
                        timeUpdated = False
                        idUpdated = False
                        res = list(itertools.islice(sync, start, end, None))
                        for index, s in enumerate(res):
                            rTag = "HTTP_REQUEST_POST"
                            sTag = "status :  200"
                            coTag = "correlationId" 
                            if rTag in s and timeUpdated == False:
                                df.at[dfIndex, cWakeupPNSSent] = s[18:31]
                                # print(s[19:31])
                                timeUpdated = True
                            if sTag in s: 
                                start = True
                                # print(start)
                            if coTag in s:
                                coId = s[s.index(coTag) + len(coTag) + 4:len(s)-1]
                                # print("Updating df: ", dfIndex, coId)
                                df.at[dfIndex, cCoId] = coId
                                dCoId[dfIndex] = coId
                                idUpdated = True
                            if index == len(res) - 1:
                                dfIndex += 1
                                # print("Increased index to: ", dfIndex)
                                
                        break
                break
                    



========== Step 3 & 4 ==========
--- Reading file: ../feedback-analysis/logs/desktop/sync.1.txt
2626e270-8293-11ee-8ae9-a9a9eeb63267
643ac2c0-8293-11ee-8ae9-a9a9eeb63267
7678b690-8293-11ee-8ae9-a9a9eeb63267
a065bed0-8293-11ee-8ae9-a9a9eeb63267
1420ab00-8294-11ee-8ae9-a9a9eeb63267
--- Reading file: ../feedback-analysis/logs/desktop/sync.txt
469fc2f0-8294-11ee-8ae9-a9a9eeb63267
99be2490-8294-11ee-8ae9-a9a9eeb63267
e5f8c2c0-8294-11ee-8ae9-a9a9eeb63267
074c4140-8295-11ee-8ae9-a9a9eeb63267


In [37]:
# Step 5 & 6
print("========== Step 5 & 6 ==========")
dPNS = {}
with open(logOTTDir + "mav_sdk.txt") as file:
    sdk = [line.rstrip() for line in file]
    start = 0
    end = 0
    for index, coId in dCoId.items():
        print(coId)
        pnsLog = list(filter(lambda x: x.find(coId) != -1, sdk))
        if len(pnsLog) == 0: 
            df.at[index, cWakeupPNSReceived] = "Not Received " + str(failed) 
            dCoId[index] = -1
            df.at[index, cSMSStatus] = failed
            continue
        else:
            df.at[index, cWakeupPNSReceived] = pnsLog[0][12:24]
            dPNS[index] = pnsLog[0][12:24]
            start = sdk.index(pnsLog[0])
            end = start + 200

        # Step 6
        registerTag = "MavNsdHelper onServiceRegistered"
        # print(start)
        # print(end)
        isServiceRegistered = False
        for l in list(itertools.islice(sdk , start, end, None)):
            if registerTag in l:
                isServiceRegistered = True
                registerTimeStamp = l[12:24]
                registerTimeStampLength = len(registerTimeStamp)
                # key = registerTimeStamp[0: registerTimeStampLength - 4]
                # for item in dPNS.items():
                    # if key in item[1]:
                        # df.at[item[0], cPacketBroadcast] = registerTimeStamp
                df.at[index, cPacketBroadcast] = registerTimeStamp
                # print("Update timestamp: ", registerTimeStamp)
                tag1 = "name: "
                tag2 = ", type"
                hashKey = l[l.index(tag1) + len(tag1):l.index(tag2)]
                # print("Update hashKey: ", hashKey)
                # print(l)
                # print("Getting hash: " + hashKey)
                dCoId[index] = (hashKey, coId)
                # print("Update dCoID: ", hashKey, coId)
                break
        if isServiceRegistered == False:
             df.at[index, cPacketBroadcast] = "MDNS Service is not registered \u2717"
             dCoId[index] = ('', coId)

========== Step 5 & 6 ==========
20f15a67-1f0f-a40c-3afd-382654771445
54dab099-aac8-3368-252b-aa6a801fec8b
5636a032-124c-150f-ac4f-109d51a3ee24
22519b1b-a48b-b8cd-284a-e8c81cc5cd6d
fa22fef3-fb13-0f73-4caa-3c2d50b03c89
033bf366-f476-6ca4-196f-9afc811658ab
81b5bfc9-c3e7-ab85-6843-698996636ee3
dcd5087c-6234-688a-ad05-f8a0d641027a
3b87431b-6d9f-ec74-bcc1-2a358ff31753


In [38]:
# Step 7
print("========== Step 7 ==========")
for f in files_discovery:
    with open(f) as file:
        print("--- Reading file:", f)
        discovery = [line.rstrip() for line in file]
        dCoId = dict(filter(lambda x: x[1] != -1, dCoId.items()))
        for index, value in list(filter(lambda x: x != -1, dCoId.items())):
            print("Checking pair: ", value)
            expiryTag = value[1] + " :: MDNS and Bluetooth response expiry timer reached"
            foundExpiry = list(filter(lambda x: expiryTag in x, discovery))
            if len(foundExpiry) > 0:
                print("    > expired pair: ", value)
                df.at[index, cPacketReceivedStatus] = str(failed)
                df.at[index, cPacketReceivedTime] = "Expiry timer reached at DC: " + foundExpiry[0][18:30] 
                df.at[index, cSMSStatus] = failed
                dCoId[index] = -1
                continue
            else:
                for line in discovery:
                    # Searching with hash key
                    if (value[0] != "" and value[0] in line):
                        print("    > by hashKey: ", value[0])
                        # print("[Found]", line)
                        df.at[index, cPacketReceivedTime] = line[11:23]
                        df.at[index, cSMSStatus] = success
                        df.at[index, cPacketReceivedStatus] = success

                        try:
                            start = _datetime(df.iloc[index][cSMSSent], dateTotalTimeFormatter)
                            end = _datetime(df.iloc[index][cPacketReceivedTime], dateTotalTimeFormatter)
                            df.at[index, cTotalTime] = (end - start).total_seconds()
                        except:
                            print("Error parsing total time: ", line)
                        dCoId[index] = -1
                        continue
                    
                    # Searching with corelation id
                    elif (value[1] != "" and value[1] in line and "Time taken" in line):
                        print("    > corelationId: ", value[1])
                        # print("[Found]", line)
                        df.at[index, cPacketReceivedTime] = line[18:30] + " (Bluetooth)"
                        df.at[index, cSMSStatus] = success
                        df.at[index, cPacketReceivedStatus] = success

                        tag = "Time taken ::"
                        totalTime = line[line.index(tag) + len(tag):len(line)]
                        # print(totalTime)
                        df.at[index, cTotalTime] = line[line.index(tag) + len(tag) + 1:len(line)]
                        dCoId[index] = -1
                        continue

                

========== Step 7 ==========
--- Reading file: ../feedback-analysis/logs/desktop/discovery.1.txt
Checking pair:  ('linkM9990#9j5tQpmDnLHLRB+DAC/LIAUv0pc=#CMWIFI', '5636a032-124c-150f-ac4f-109d51a3ee24')
Checking pair:  ('linkM7293#b9G9JhkaQoVtiCRbL4kcReGJ2Tc=#CMWIFI', '22519b1b-a48b-b8cd-284a-e8c81cc5cd6d')
Checking pair:  ('linkM9556#MRCbGX22GHWP9lZPvR0b5AF7rlk=#CMWIFI', 'fa22fef3-fb13-0f73-4caa-3c2d50b03c89')
Checking pair:  ('linkM2493#kIKdNEQU/vFFjHV5xRUhFRhJDos=#CMWIFI', '033bf366-f476-6ca4-196f-9afc811658ab')
Checking pair:  ('linkM2824#dNRsQYMV8kosYKKPTZkmFhSj1NU=#CMWIFI', '81b5bfc9-c3e7-ab85-6843-698996636ee3')
Checking pair:  ('', 'dcd5087c-6234-688a-ad05-f8a0d641027a')
Checking pair:  ('', '3b87431b-6d9f-ec74-bcc1-2a358ff31753')
--- Reading file: ../feedback-analysis/logs/desktop/discovery.txt
Checking pair:  ('linkM9990#9j5tQpmDnLHLRB+DAC/LIAUv0pc=#CMWIFI', '5636a032-124c-150f-ac4f-109d51a3ee24')
    > by hashKey:  linkM9990#9j5tQpmDnLHLRB+DAC/LIAUv0pc=#CMWIFI
Checking pair:

In [39]:
df
# df.to_csv("LogAnalysis.csv")

,Date,Co-Relation ID,SMS Sent from UI (Desktop),Wake up PNS Sent to WRG (Desktop),Wake up PNS Received (OTT),Starting broadcast MDNS/BLE Packet (OTT),Packet Received (Desktop) ❌ or ✅,Packet Received Time (Desktop),Total Round-Trip Time,Overall SMS Status ❌ or ✅
0,2023-11-14,20f15a67-1f0f-a40c-3afd-382654771445,11:11:40.619,11:11:40.696,Not Received ❌,NaN,NaN,NaN,NaN,❌
1,2023-11-14,54dab099-aac8-3368-252b-aa6a801fec8b,11:13:24.841,11:13:24.845,Not Received ❌,NaN,NaN,NaN,NaN,❌
2,2023-11-14,5636a032-124c-150f-ac4f-109d51a3ee24,11:13:55.448,11:13:55.450,11:13:55.456,11:13:56.469,✅,11:13:56.081,0.633,✅
3,2023-11-14,22519b1b-a48b-b8cd-284a-e8c81cc5cd6d,11:15:05.764,11:15:05.791,11:15:06.966,11:15:07.989,✅,11:15:07.561,1.797,✅
4,2023-11-14,fa22fef3-fb13-0f73-4caa-3c2d50b03c89,11:18:19.948,11:18:19.954,11:18:19.896,11:18:20.812,✅,11:18:24.339 (Bluetooth),4289.5,✅
5,2023-11-14,033bf366-f476-6ca4-196f-9afc811658ab,11:19:44.668,11:19:44.673,11:19:45.716,11:19:46.536,✅,11:19:50.815 (Bluetooth),5953.5,✅
6,2023-11-14,81b5bfc9-c3e7-ab85-6843-698996636ee3,11:22:04.117,11:22:04.123,11:22:04.052,11:22:04.896,❌,Expiry timer reached at DC: 11:22:24.290,NaN,❌
7,2023-11-14,dcd5087c-6234-688a-ad05-f8a0d641027a,11:24:12.008,11:24:12.013,11:24:12.558,MDNS Service is not registered ✗,❌,Expiry timer reached at DC: 11:24:32.172,NaN,❌
8,2023-11-14,3b87431b-6d9f-ec74-bcc1-2a358ff31753,11:25:07.922,11:25:07.925,11:25:08.620,MDNS Service is not registered ✗,✅,11:25:19.905 (Bluetooth),11810.5,✅
